# Run stargeo queries for hetio-ind diseases

In [1]:
import logging
import os
import gzip

import pandas
import easydict

import starapi.analysis
import starapi.conf

In [2]:
starapi.analysis.logger.propagate = False
starapi.conf.configure('data')

In [3]:
query_df = pandas.read_table('data/queries.tsv')
query_df.head()

,slim_id,slim_name,case_query,control_query
0,DOID:0050156,idiopathic pulmonary fibrosis,IPF == 'IPF',IPF_control == 'IPF_control'
1,DOID:0050741,alcohol dependence,alcoholism == 'alcoholism',alcoholism_control == 'alcoholism_control'
2,DOID:0050742,nicotine dependence,Smoker == 'Smoker',Smoker_control == 'Smoker_control'
3,DOID:1024,leprosy,borderline_leprosy == 'borderline_leprosy' or ...,leprosy_control == 'leprosy_control'
4,DOID:10283,prostate cancer,PC == 'PC',PC_control == 'PC_control' or PC_tissue_contro...


In [4]:
errors = list()

for i, row in query_df.iterrows():
    print i, row.slim_id, row.slim_name

    name = row.slim_id.replace(':', '_')
    params = easydict.EasyDict(
        analysis_name = name,
        case_query = row.case_query,
        control_query = row.control_query,
        modifier_query = '',
        min_samples = 3,
    )
    
    directory = os.path.join('data', 'doslim', params.analysis_name)
    if not os.path.isdir(directory):
        os.mkdir(directory)

    logfile = logging.FileHandler(os.path.join(directory, 'log.txt'))
    logfile.setLevel(logging.DEBUG)
    starapi.analysis.logger.addHandler(logfile)
    try:
        sample_df, fold_df, balanced_perm_df, perm_df = starapi.analysis.perform_analysis(params)
        for df in fold_df, balanced_perm_df:
            if df is not None:
                df.reset_index(inplace=True)
        
    except Exception as e:
        sample_df, fold_df, balanced_perm_df, perm_df = None, None, None, None
        errors.append({'slim_id': row.slim_id, 'name': name, 'error': e})
        print(e)
    starapi.analysis.logger.removeHandler(logfile)
    
    # write files
    files = [
        (sample_df, 'samples.tsv', open),
        (fold_df, 'fold_change.tsv.gz', gzip.open),
        (balanced_perm_df, 'balanced_permutation.tsv.gz', gzip.open),
    ]
    for df, filename, opener in files:
        if df is None:
            continue
        path = os.path.join(directory, filename)
        with opener(path, 'wt') as write_file:
            df.to_csv(write_file, index=False, sep='\t', float_format='%.5g')


Started DOID_0050156 analysis
/home/dhimmels/anaconda3/envs/stargeo/lib/python2.7/site-packages/pandas/core/indexing.py:115: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)
382.78 ms in get_analysis_df("IPF == 'IPF'", "IPF_control == 'IPF_c..., '')
383.63 ms in Loading dataframe for DOID_0050156
Matching sources: 12
Stats: 12 sources, 12 series, 8 platforms, 679 samples
Loading data and calculating fold change for DOID_0050156
Loading data for GSE10667, id = 179
324.39 ms in get_probes(10)
  3.86 s in load_gse( series_id platform_id..., 179, False)
/home/dhimmels/Documents/serg/rephetio/construct/star_api/starapi/analysis.py:164: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

0 DOID:0050156 idiopathic pulmonary fibrosis
1

Started DOID_0050741 analysis
296.07 ms in get_analysis_df("alcoholism == 'alcoho..., "alcoholism_control ==..., '')
296.93 ms in Loading dataframe for DOID_0050741
Matching sources: 8
Stats: 8 sources, 8 series, 8 platforms, 435 samples
Loading data and calculating fold change for DOID_0050741
Loading data for GSE10356, id = 95
 32.20 ms in get_probes(36)
  3.39 s in load_gse( series_id platform_id..., 95, False)
 40.47 ms in get_gene_fold_change(<Gse GSE10356>, False, 0, None)
Loading data for GSE20568, id = 2733
 32.07 ms in get_probes(689)
474.04 ms in load_gse( series_id platform_id..., 2733, False)
 53.62 ms in get_gene_fold_change(<Gse GSE20568>, False, 0, None)
Loading data for GSE29555, id = 5002
284.94 ms in get_probes(389)
  3.55 s in load_gse( series_id platform_id..., 5002, False)
  1.02 s in get_gene_fold_change(<Gse GSE29555>, False, 0, None)
Loading data for GSE44456, id = 8480
246.75 ms in get_probes(283)
  2.14 s in load_gse( series_id platform_id..., 8480, False)
449

 DOID:0050741 alcohol dependence
2

Started DOID_0050742 analysis
317.50 ms in get_analysis_df("Smoker == 'Smoker'", "Smoker_control == 'Sm..., '')
318.34 ms in Loading dataframe for DOID_0050742
Matching sources: 22
Excluded 3 sources as single-class
Stats: 19 sources, 17 series, 6 platforms, 1423 samples
Loading data and calculating fold change for DOID_0050742
Loading data for GSE10006, id = 4
300.47 ms in get_probes(4)
  7.76 s in load_gse( series_id platform_id..., 4, False)
  1.28 s in get_gene_fold_change(<Gse GSE10006>, False, 0, None)
Loading data for GSE11784, id = 450
307.23 ms in get_probes(4)
 13.39 s in load_gse( series_id platform_id..., 450, False)
  2.20 s in get_gene_fold_change(<Gse GSE11784>, False, 0, None)
Loading data for GSE11906, id = 487
302.13 ms in get_probes(4)
 18.47 s in load_gse( series_id platform_id..., 487, False)
  3.18 s in get_gene_fold_change(<Gse GSE11906>, False, 0, None)
Loading data for GSE19407, id = 2430
301.10 ms in get_probes(4)
 11.31 s in load_gse( series_id platform_id...

...

 DOID:9744 type 1 diabetes mellitus
64

Started DOID_986 analysis
175.31 ms in get_analysis_df("alopecia_areata == 'a..., "alopecia_areata_contr..., '')
176.15 ms in Loading dataframe for DOID_986
Matching sources: 2
Stats: 2 sources, 2 series, 1 platforms, 17 samples
Loading data and calculating fold change for DOID_986
Loading data for GSE45512, id = 8684
299.20 ms in get_probes(4)
  1.87 s in load_gse( series_id platform_id..., 8684, False)
422.44 ms in get_gene_fold_change(<Gse GSE45512>, False, 0, None)
Loading data for GSE58573, id = 10697
301.78 ms in get_probes(4)
  1.59 s in load_gse( series_id platform_id..., 10697, False)
366.88 ms in get_gene_fold_change(<Gse GSE58573>, False, 0, None)
  4.35 s in Load/fold for DOID_986
Meta-Analyzing DOID_986
 48.44 s in meta analysis of real data for DOID_986
 55.11 ms in meta analysis of permutations for DOID_986
 19.07 mks in get_balanced_permutations({'analysis_name': 'DOI...,  C H H_lower H_upper I..., Empty DataFrame Column...)
 48.58 s in Meta analysis for DOID_986
DONE D

 DOID:986 alopecia areata
65

Started DOID_9970 analysis
369.50 ms in get_analysis_df("obesity == 'obesity'", "obesity_control == 'o..., '')
370.46 ms in Loading dataframe for DOID_9970
Matching sources: 43
Excluded 23 sources as single-class
Stats: 20 sources, 14 series, 16 platforms, 561 samples
Loading data and calculating fold change for DOID_9970
Loading data for GSE12050, id = 521
308.65 ms in get_probes(332)
  2.33 s in load_gse( series_id platform_id..., 521, False)
437.75 ms in get_gene_fold_change(<Gse GSE12050>, False, 0, None)
Loading data for GSE15524, id = 1461
215.90 ms in get_probes(500)
  1.47 s in load_gse( series_id platform_id..., 1461, False)
255.22 ms in get_gene_fold_change(<Gse GSE15524>, False, 0, None)
Loading data for GSE2508, id = 3892
108.61 ms in get_probes(7)
109.37 ms in get_probes(8)
136.71 ms in get_probes(13)
107.08 ms in get_probes(14)
134.10 ms in get_probes(15)
170.68 ms in get_probes(16)
  5.15 s in load_gse( series_id platform_id..., 3892, False)
770.97 ms in get_gene_fold_ch

 DOID:9970 obesity


In [5]:
error_df = pandas.DataFrame(errors)
error_df = query_df.merge(error_df)
error_df

,slim_id,slim_name,case_query,control_query,error,name
0,DOID:11054,urinary bladder cancer,bladder_cancer == 'bladder_cancer',bladder_cancer_control == 'bladder_cancer_cont...,'Int64Index' object has no attribute 'labels',DOID_11054
1,DOID:11714,gestational diabetes,GDM == 'GDM',GDM_control == 'GDM_control','mygene_sym',DOID_11714
2,DOID:12849,autistic disorder,autism == 'autism',autism_control == 'autism_control',zero-size array to reduction operation maximum...,DOID_12849
3,DOID:14221,metabolic syndrome X,MetS == 'MetS',MetS_Control == 'MetS_Control','Int64Index' object has no attribute 'labels',DOID_14221
4,DOID:2998,testicular cancer,testicular_cancer == 'testicular_cancer',testicular_cancer_control == 'testicular_cance...,'Int64Index' object has no attribute 'labels',DOID_2998
5,DOID:3083,chronic obstructive pulmonary disease,COPD == 'COPD',COPD_control == 'COPD_control',"Can't find matrix file for series 6962, platfo...",DOID_3083
6,DOID:4606,bile duct cancer,bile_duct_cancer == 'bile_duct_cancer',bile_duct_cancer_control == 'bile_duct_cancer_...,'Int64Index' object has no attribute 'labels',DOID_4606


In [6]:
error_df.to_csv('data/errors.csv', index=False)